# GEO1003 ~ Wi-Fi Fingerprinting

December 2023

Work by Tom Barbian (5100704), Der Derian Auliyaa Bainus (5941342), Corné Snoeij (5174473), Javier Martínez (6055613)

### Importing libraries

In [1]:
import pandas as pd
import numpy as np

### Loading the data

### And more ... 

In [2]:
print("hello world")

hello world
